<a href="https://colab.research.google.com/github/100495773/Grupo05_Practica1/blob/main/notebook_modelo_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modelo final

En este notebook implementaremos el modelo final escogido con sus respectivos hiperparámetros en base al tarbajo realizado en el otro notebook, para finalmente guardar el modelo y sus predicciones en sus respectivos documentos a entregar.

En este apartado, ya que lo que se va a realizar es la implementación de un modelo ya realizado con anterioridad en el otro notebook, las explicaciones dadas en este documento serán breves.

Cargamos el modelo.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving attrition_availabledata_05.csv to attrition_availabledata_05.csv


Hacemos los debidos imports.

In [2]:
%matplotlib inline
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Preprocesamiento y modelización
# ==============================================================================
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, confusion_matrix, balanced_accuracy_score
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn import tree
from sklearn.model_selection import train_test_split, cross_val_score, KFold, RandomizedSearchCV, GridSearchCV
from sklearn import metrics
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB

# Bibliotecas varias
# ==============================================================================
import time
import joblib

Introducimos el dataset en un formato manipulable por Python.

In [3]:
df = pd.read_csv('attrition_availabledata_05.csv')

Separamos la variable objetivo del resto de variables.

In [4]:
X = df.drop(columns=["Attrition"])
y = df["Attrition"]

Quitamos las variables constantes.

In [5]:
X.drop(['EmployeeCount', 'EmployeeID', 'Over18', 'StandardHours'], axis=1, inplace=True)

Codificamos la variable objetivo.

In [6]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

Separamos las variables de entrada en categóricas y numéricas.

In [7]:
cat_cols = X.select_dtypes(include="object").columns
num_cols = X.select_dtypes(exclude="object").columns

Preprocesamos las variables categóricas.

In [8]:
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ('pca', PCA(n_components=25, random_state=100495773))
])

Preprocesamos las variables numéricas.

In [9]:
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

Concatenamos el preprocesado de las variables numéricas y categóricas.

In [10]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols)
    ])

Definimos el clasificador a usar, en nuestro caso SVM's (SVC), con sus respectivos mejores hiperparámetros.

In [11]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', SVC(C=1000, gamma='auto', kernel='rbf', random_state=100495773))])

Entrenamos el modelo de clasificación con las variables de entrada y de salida.

In [12]:
clf.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  Index(['hrs', 'absences', 'JobInvolvement', 'PerformanceRating',
       'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'Age',
       'DistanceFromHome', 'Education', 'JobLevel', 'MonthlyIncome',
       'Nu...
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('pca',
                                                                   PCA(n_components=25,
                                                                       random_state=100495773))]),
                                                  Index(['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole',
       'MaritalStatus'],
      dtype='object'))])),
                ('classifier',
                 SVC(C=1000, gamma='auto', random_state=100495773))])

Una vez cargado el modelo final, y entrenado con los datos del "availabledata", cargaremos el dataset "competition" para que se hagan evaluaciones en base a este.

Cargamos el dataset de las variables a procesar para sacar sus predicciones de la variable objetivo, 'competition'.

In [13]:
from google.colab import files
uploaded = files.upload()

Saving attrition_competition_05.csv to attrition_competition_05.csv


Cargamos el dataset en un formato manipulable por Python.

In [14]:
df = pd.read_csv('attrition_competition_05.csv')

In [15]:
y_pred = clf.predict(df)
y_pred[:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0])

Por lo ofrecido en primera instancia en las predicciones, observamos que la tasa tanto de negativos como de positivos es similar a la que esperábamos. Para mayor precisión, vamos a comparar la proporción de ambas de forma porcentual con las del dataset original.

In [22]:
df = pd.DataFrame({'Attrition': y_pred})
print(df['Attrition'].value_counts())

print()

print(df['Attrition'].value_counts()/df['Attrition'].count())

Attrition
0    1236
1     234
Name: count, dtype: int64

Attrition
0    0.840816
1    0.159184
Name: count, dtype: float64


Como podemos observar, comparando estas cifras con las ofrecidas por el dataset 'availabledata', las proporciones de ambas son similares. Mientras que esta ofrece un 84,1% de negativos y un 15,9% de positivos, el dataset original da unas cifras de 83,8% y 16,2% respectivamente. Dados estos resultados podemos afirmar que las predicciones ofrecidas por el modelo son bastante precisas.

Por último, guardaremos el modelo en un archivo llamado "modelo_final.plk". Y las predicciones serán guardadas en un modelo llamado "predicciones.csv"

In [16]:
joblib.dump(clf, 'modelo_final.pkl')

['modelo_final.pkl']

In [21]:
df.to_csv('predicciones.csv', index=False)

Como conclusión, hemos obtenido un modelo que, pese a ofrecer una predicción bastante baja en las pruebas de testeo durante su modelozación, al final parece ofrecer resultados bastante satisfactorios. Aunque dichos resultados no tienen que significar que el modelo predice correctamente la salida en base a los datos de entrada introducidos, por las pruebas que somos capaces de realizar a priori parece que así es.